In [38]:
import sys
import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
from scipy.sparse.linalg.eigen.arpack import eigsh

In [55]:
def sample_mask(idx, l):
    """Create mask."""
    mask = np.zeros(l)
    mask[idx] = 1
    return np.array(mask, dtype=np.bool)

def parse_index_file(filename):
    """Parse index file."""
    index = []
    for line in open(filename):
        index.append(int(line.strip()))
    return index

In [60]:
dataset_str =  'ahu'
names = ['x', 'y', 'tx', 'ty', 'allx', 'ally', 'graph', 'test.index']
# names = ['x', 'y', 'tx', 'ty', 'allx', 'ally', 'graph']
objects = []
for i in range(len(names)):
    with open("gcn/gcn/data/ind.{}.{}".format(dataset_str, names[i]), 'rb') as f:
        if sys.version_info > (3, 0):
            objects.append(pkl.load(f, encoding='latin1'))
        else:
            objects.append(pkl.load(f))

x, y, tx, ty, allx, ally, graph, test_idx_reorder = tuple(objects)
# x, y, tx, ty, allx, ally, graph = tuple(objects)
# test_idx_reorder = parse_index_file("gcn/gcn/data/ind.{}.test.index".format(dataset_str))
test_idx_range = np.sort(test_idx_reorder)

In [61]:
print(x.shape)
print(tx.shape)
print(len(graph))

(216, 480)
(433, 480)
1440


In [62]:
features = sp.vstack((allx, tx)).tolil()
features[test_idx_reorder, :] = features[test_idx_range, :]
adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))

labels = np.vstack((ally, ty))
labels[test_idx_reorder, :] = labels[test_idx_range, :]

idx_test = test_idx_range.tolist()
idx_train = range(len(y))
idx_val = range(len(y), len(y)+500)

train_mask = sample_mask(idx_train, labels.shape[0])
val_mask = sample_mask(idx_val, labels.shape[0])
test_mask = sample_mask(idx_test, labels.shape[0])

y_train = np.zeros(labels.shape)
y_val = np.zeros(labels.shape)
y_test = np.zeros(labels.shape)
y_train[train_mask, :] = labels[train_mask, :]
y_val[val_mask, :] = labels[val_mask, :]
y_test[test_mask, :] = labels[test_mask, :]

In [63]:
adjlist = adj.toarray()
len(adjlist[0])

1440

In [33]:
print(type(x))
print(type(tx))
print(type(allx))
print(type(y))
print(type(ty))
print(type(ally))
print(type(graph))
print(type(test_idx_range))

<class 'scipy.sparse.csr.csr_matrix'>
<class 'scipy.sparse.csr.csr_matrix'>
<class 'scipy.sparse.csr.csr_matrix'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'collections.defaultdict'>
<class 'numpy.ndarray'>
